# Import modules


In [21]:
import numpy as np
from pyfluids import Fluid, FluidsList, Input

from thermosys.processes import (
    heat_to_temperature,
    turbine_to_enthalpy,
    turbine_to_pressure,
)
from thermosys.processes.gas import compress_to
from thermosys.processes.vapor import condense_to_pressure, pump_to_pressure
from thermosys.services.energy import energy_balance
from thermosys.services.misc import print_states
from thermosys.services.montecarlo import get_random_value, plot_monte_carlo
from thermosys.services.units import bar_to_pascal, pascal_to_bar

# Constants


In [22]:
GAS_MASS_FLOW_PER_SYSTEM = 450  # kg/s
NUMBER_OF_GAS_SYSTEMS = 2  # -

GAS_INLET_PRESSURE = bar_to_pascal(1)  # Pa
GAS_INLET_TEMPERATURE = 30  # C
GAS_MASS_FLUX = 450  # kg/s
GAS_COMPRESSOR_EFFICIENCY = 0.8  # -
GAS_COMPRESSION_RATIO = 25  # -
GAS_TURBINE_C_EFFICIENCY = 0.8  # -
GAS_COMBUSTION_CHAMBER_1_TEMPERATURE = 1000  # C
GAS_TURBINE_P_EFFICIENCY = 0.85  # -
GAS_COMBUSTION_CHAMBER_2_TEMPERATURE = 710  # C

VAPOR_DEAERATOR_PRESSURE = bar_to_pascal(5)  # Pa
VAPOR_TURBINE_EFFICIENCY = 0.9  # -
VAPOR_CONDENSER_PRESSURE = bar_to_pascal(0.1)  # Pa
VAPOR_PUMP_EFFICIENCY = 0.9999  # -
VAPOR_DEAERATOR_TEMPERATURE = 120  # C

AMBIENT_PRESSURE = bar_to_pascal(1)  # Pa
AMBIENT_TEMPERATURE = 30  # C

RECOVERY_BOILER_DELTA_TSA = 25  # C
RECOVERY_BOILER_DELTA_TP = 10  # C
RECOVERY_BOILER_DELTA_TE = 10  # C

OPTIMIZATION_ITERATION_COUNT = 1000  # -
MAX_VAPOR_INLET_PRESSURE = bar_to_pascal(200)  # Pa

# Brayton Gas Cycle


In [23]:
state_1 = Fluid(FluidsList.Air).with_state(
    Input.pressure(GAS_INLET_PRESSURE),
    Input.temperature(GAS_INLET_TEMPERATURE),
)

state_2 = compress_to(
    inlet_state=state_1,
    efficiency=GAS_COMPRESSOR_EFFICIENCY,
    compression_ratio=GAS_COMPRESSION_RATIO,
)

state_3 = heat_to_temperature(
    inlet_state=state_2,
    outlet_temperature=GAS_COMBUSTION_CHAMBER_1_TEMPERATURE,
)

energy_balance_compressor = energy_balance(state_1, state_2)
state_4 = turbine_to_enthalpy(
    inlet_state=state_3,
    efficiency=GAS_TURBINE_C_EFFICIENCY,
    energy_balance=energy_balance_compressor,
)

state_5 = heat_to_temperature(
    inlet_state=state_4,
    outlet_temperature=GAS_COMBUSTION_CHAMBER_2_TEMPERATURE,
)

state_6 = turbine_to_pressure(
    inlet_state=state_5,
    efficiency=GAS_TURBINE_P_EFFICIENCY,
    outlet_pressure=GAS_INLET_PRESSURE,
)

gas_states = [state_1, state_2, state_3, state_4, state_5, state_6]

gas_turbine_specific_work = energy_balance(
    gas_states[3], gas_states[2]
) + energy_balance(gas_states[5], gas_states[4])
gas_compressor_specific_work = energy_balance(gas_states[0], gas_states[1])
gas_net_specific_work = (
    gas_turbine_specific_work - gas_compressor_specific_work
)
gas_specific_heat_in = energy_balance(
    gas_states[1], gas_states[2]
) + energy_balance(gas_states[3], gas_states[4])

gas_efficiency = gas_net_specific_work / gas_specific_heat_in
gas_bwr = gas_turbine_specific_work / gas_compressor_specific_work

print("BRAYTON CYCLE")
print_states(gas_states)


print("\n")

print(
    f"Gas turbine specific work: {gas_turbine_specific_work * 1e-3:.2f} kJ/kg"
)
print(
    f"Gas compressor specific work: {gas_compressor_specific_work * 1e-3:.2f} kJ/kg"
)
print(f"Gas net specific work: {gas_net_specific_work * 1e-3:.2f} kJ/kg")
print(f"Gas specific heat in: {gas_specific_heat_in * 1e-3:.2f} kJ/kg")
print(f"Gas thermal efficiency: {gas_efficiency * 100:.2f} %")
print(f"Gas BWR: {gas_bwr * 100:.2f} %")

print("\n")

BRAYTON CYCLE
1 - Air: 1.00 bar, 30.00 C, 429.47 kJ/kg
2 - Air: 25.00 bar, 573.80 C, 1001.22 kJ/kg
3 - Air: 25.00 bar, 1000.00 C, 1492.34 kJ/kg
4 - Air: 1.72 bar, 501.44 C, 920.59 kJ/kg
5 - Air: 1.72 bar, 710.00 C, 1153.36 kJ/kg
6 - Air: 1.00 bar, 602.06 C, 1031.68 kJ/kg


Gas turbine specific work: 693.43 kJ/kg
Gas compressor specific work: 571.75 kJ/kg
Gas net specific work: 121.68 kJ/kg
Gas specific heat in: 723.89 kJ/kg
Gas thermal efficiency: 16.81 %
Gas BWR: 121.28 %




# Optimization of the Rankine Vapor Cycle and Recovery Boiler


In [24]:
pressures = np.array([])
system_efficiencies = np.array([])

for _ in range(OPTIMIZATION_ITERATION_COUNT):
    # Optimization variables:
    inlet_pressure = get_random_value(
        min_value=VAPOR_DEAERATOR_PRESSURE, max_value=MAX_VAPOR_INLET_PRESSURE
    )

    # Solving the Rankine Cycle:
    state_7 = Fluid(FluidsList.Water).with_state(
        Input.pressure(inlet_pressure),
        Input.temperature(VAPOR_DEAERATOR_TEMPERATURE),
    )

    state_1 = heat_to_temperature(
        inlet_state=state_7,
        outlet_temperature=gas_states[-1].temperature
        - RECOVERY_BOILER_DELTA_TSA,
    )

    state_2 = turbine_to_pressure(
        inlet_state=state_1,
        efficiency=VAPOR_TURBINE_EFFICIENCY,
        outlet_pressure=VAPOR_DEAERATOR_PRESSURE,
    )

    state_3 = turbine_to_pressure(
        inlet_state=state_2,
        efficiency=VAPOR_TURBINE_EFFICIENCY,
        outlet_pressure=VAPOR_CONDENSER_PRESSURE,
    )

    state_4 = condense_to_pressure(
        inlet_state=state_3,
    )

    state_5 = pump_to_pressure(
        inlet_state=state_4,
        efficiency=VAPOR_PUMP_EFFICIENCY,
        outlet_pressure=VAPOR_DEAERATOR_PRESSURE,
    )

    state_6 = state_5.with_state(
        Input.pressure(state_5.pressure),
        Input.entropy(state_7.entropy),
    )

    vapor_states = [
        state_1,
        state_2,
        state_3,
        state_4,
        state_5,
        state_6,
        state_7,
    ]

    vapor_turbine_specific_work = energy_balance(
        vapor_states[1], vapor_states[0]
    ) + energy_balance(vapor_states[2], vapor_states[1])
    vapor_pump_specific_work = energy_balance(
        vapor_states[3], vapor_states[4]
    ) + energy_balance(vapor_states[5], vapor_states[6])
    vapor_net_specific_work = (
        vapor_turbine_specific_work - vapor_pump_specific_work
    )
    vapor_specific_heat_in = energy_balance(vapor_states[-1], vapor_states[0])

    vapor_efficiency = vapor_net_specific_work / vapor_specific_heat_in

    # Solving the Recovery Boiler:
    gas_mass_flow = GAS_MASS_FLOW_PER_SYSTEM / NUMBER_OF_GAS_SYSTEMS

    state_8_evaporator = Fluid(FluidsList.Water).two_phase_point_at_pressure(
        pressure=vapor_states[0].pressure,
        quality=0,
    )

    state_12 = Fluid(FluidsList.Water).two_phase_point_at_pressure(
        pressure=vapor_states[0].pressure,
        quality=100,
    )

    temperature_a = state_8_evaporator.temperature + RECOVERY_BOILER_DELTA_TP
    state_a = Fluid(FluidsList.Air).with_state(
        Input.pressure(gas_states[-1].pressure),
        Input.temperature(temperature_a),
    )

    temperature_8_economizer = (
        state_8_evaporator.temperature - RECOVERY_BOILER_DELTA_TE
    )
    state_8_economizer = Fluid(FluidsList.Water).with_state(
        Input.pressure(vapor_states[0].pressure),
        Input.temperature(temperature_8_economizer),
    )

    heat_evaporator_sup = (
        energy_balance(state_a, gas_states[-1]) * gas_mass_flow
    )

    mass_flow_evaporator = heat_evaporator_sup / (
        energy_balance(vapor_states[0], state_8_evaporator)
    )

    enthalpy_b = gas_states[
        -1
    ].enthalpy - mass_flow_evaporator / gas_mass_flow * (
        energy_balance(state_8_economizer, vapor_states[-1])
    )

    state_b = state_a.with_state(
        Input.enthalpy(enthalpy_b), Input.pressure(gas_states[-1].pressure)
    )

    outlet_gas_enthalpy = (
        state_a.enthalpy
        - mass_flow_evaporator
        / gas_mass_flow
        * (energy_balance(state_8_economizer, vapor_states[-1]))
    )

    outlet_gas_state = state_b.with_state(
        Input.pressure(gas_states[-1].pressure),
        Input.temperature(state_b.temperature),
    )

    state_ambient = Fluid(FluidsList.Air).with_state(
        Input.pressure(AMBIENT_PRESSURE),
        Input.temperature(AMBIENT_TEMPERATURE),
    )

    enthalpy_7_gas = state_a.enthalpy

    gas_heat_superheater = mass_flow_evaporator * energy_balance(
        state_8_evaporator, vapor_states[-1]
    )
    gas_heat_evaporator = mass_flow_evaporator * energy_balance(
        state_b, state_a
    )
    gas_heat_economizer = mass_flow_evaporator * energy_balance(
        state_a, outlet_gas_state
    )

    vapor_heat_evaporator = mass_flow_evaporator * energy_balance(
        state_12, state_8_evaporator
    )

    boiler_efficiency = energy_balance(
        gas_states[-1], outlet_gas_state
    ) / energy_balance(gas_states[-1], state_ambient)

    pressures = np.append(pressures, inlet_pressure)
    system_efficiencies = np.append(
        system_efficiencies,
        gas_efficiency
        + vapor_efficiency * boiler_efficiency * (1 - gas_efficiency),
    )

In [25]:
max_efficiency = np.max(system_efficiencies)
pressure_at_max_efficiency = pressures[system_efficiencies == max_efficiency]

print("\n")

print(f"Max efficiency: {max_efficiency * 100:.2f} %")
print(
    f"Vapor inlet pressure for max efficiency: {pascal_to_bar(pressure_at_max_efficiency[0]):.2f} bar"
)

plot_monte_carlo(pascal_to_bar(pressures), system_efficiencies)



Max efficiency: 27.46 %
Vapor inlet pressure for max efficiency: 199.98 bar
